In [1]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # 학습, 테스트set 구분
from sklearn.tree import export_graphviz # tree 시각화를 위해
import graphviz # tree 시각화
from sklearn.metrics import f1_score# 성능지표를 계산하기 위해 import
from sklearn.model_selection import cross_val_score, cross_validate # 교차검증

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/LGaimers/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/LGaimers/test.csv')

In [3]:
print(train_df.shape)
print(test_df.shape)

(598, 2881)
(310, 2879)


## 사용 함수 정리

In [4]:
# 모두 결측치인 변수 제거
def remove_all_nan(dataframe):
  col_list = dataframe.columns
  nan_list = []
  nan_cnt = []
  nan_col = []
  full_list = []

  for col in col_list:
    if dataframe[col].isnull().sum() == 0:
      full_list.append(col)
      continue
    nan_list.append([col,dataframe[col].isnull().sum()])
    nan_cnt.append(dataframe[col].isnull().sum())
    nan_col.append(col)

  del_col = []
  for nan in nan_list:
    if nan[1] == len(dataframe):
      del_col.append(nan[0])
  
  return dataframe.drop(columns=del_col)

In [5]:
'''값이 1개 존재하면 제거'''

def remove_one_value(dataframe):
  del_col = []
  col_list = dataframe.columns
  for col in col_list:
      if dataframe[col].nunique()==1 :
          del_col.append(col)
  
  return dataframe.drop(columns=del_col)

## PRODUCT_CODE 별 split

In [6]:
trainA_31 = train_df[train_df['PRODUCT_CODE'] == 'A_31']
trainT_31 = train_df[train_df['PRODUCT_CODE'] == 'T_31']
trainO_31 = train_df[train_df['PRODUCT_CODE'] == 'O_31']

In [7]:
testA_31 = test_df[test_df['PRODUCT_CODE'] == 'A_31']
testT_31 = test_df[test_df['PRODUCT_CODE'] == 'T_31']
testO_31 = test_df[test_df['PRODUCT_CODE'] == 'O_31']

In [8]:
trainA_31 = remove_all_nan(trainA_31)
trainT_31 = remove_all_nan(trainT_31)
trainO_31 = remove_all_nan(trainO_31)

In [9]:
trainA_31 = remove_one_value(trainA_31)
trainT_31 = remove_one_value(trainT_31)
trainO_31 = remove_one_value(trainO_31)

## raw data 준비

x_train
- trainA_31_x
- trainT_31_x
- trainO_31_x

y_train
- trainA_31_y_r
- trainT_31_y_r
- trainO_31_y_r  

- trainA_31_y_c
- trainT_31_y_c
- trainO_31_y_c

test
- testA_31_x
- testT_31_x
- testO_31_x

In [10]:
trainA_31

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,X_128,X_129,X_130,X_131,X_132,...,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,7813.0,7813.0,NaN,NaN,0.19,...,197.286667,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,NaN,NaN,19854.0,19854.0,0.20,...,193.296552,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,7815.0,7815.0,NaN,NaN,0.19,...,179.820690,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,NaN,NaN,19856.0,19856.0,0.20,...,181.920690,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,7817.0,7817.0,NaN,NaN,0.19,...,196.393333,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,TRAIN_583,0,0.522340,2022-09-05 8:34,T050304,11920.0,11920.0,NaN,NaN,0.19,...,191.993103,181.6,394.0,371.943396,353.0,51.71,59.64,54.61,57.05,63.18
584,TRAIN_584,0,0.519519,2022-09-05 11:09,T010305,NaN,NaN,NaN,NaN,NaN,...,195.682143,184.9,466.0,448.634615,432.0,NaN,NaN,NaN,NaN,NaN
585,TRAIN_585,0,0.515214,2022-09-05 11:17,T010306,NaN,NaN,NaN,NaN,NaN,...,187.396552,176.7,472.0,450.339623,432.0,NaN,NaN,NaN,NaN,NaN
594,TRAIN_594,0,0.524022,2022-09-08 22:38,T050304,14810.0,14810.0,NaN,NaN,0.19,...,180.810345,168.7,384.0,369.811321,353.0,49.47,53.07,50.89,55.10,66.49


In [11]:
trainA_31_x = trainA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','LINE'])
trainT_31_x = trainT_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','LINE'])
trainO_31_x = trainO_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','LINE'])

In [12]:
trainA_31_x = trainA_31_x.fillna(-1)
trainT_31_x = trainT_31_x.fillna(-1)
trainO_31_x = trainO_31_x.fillna(-1)

In [13]:
testA_31_x = testA_31[trainA_31_x.columns]
testT_31_x = testT_31[trainT_31_x.columns]
testO_31_x = testO_31[trainO_31_x.columns]

In [14]:
testA_31_x = testA_31_x.fillna(-1)
testT_31_x = testT_31_x.fillna(-1)
testO_31_x = testO_31_x.fillna(-1)

In [15]:
# GCN에 넣기 위해 dataframe -> tensor 변환
trainA_31_x = torch.from_numpy(trainA_31_x.values)
trainT_31_x = torch.from_numpy(trainT_31_x.values)
trainO_31_x = torch.from_numpy(trainO_31_x.values)

testA_31_x = torch.from_numpy(testA_31_x.values)
testT_31_x = torch.from_numpy(testT_31_x.values)
testO_31_x = torch.from_numpy(testO_31_x.values)

In [16]:
trainA_31['제품상태'] = [1 if i==1 else 0 for i in trainA_31['Y_Class']]
trainT_31['제품상태'] = [1 if i==1 else 0 for i in trainT_31['Y_Class']]
trainO_31['제품상태'] = [1 if i==1 else 0 for i in trainO_31['Y_Class']]

In [17]:
trainA_31

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,X_128,X_129,X_130,X_131,X_132,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,제품상태
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,7813.0,7813.0,NaN,NaN,0.19,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,1
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,NaN,NaN,19854.0,19854.0,0.20,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,0
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,7815.0,7815.0,NaN,NaN,0.19,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,1
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,NaN,NaN,19856.0,19856.0,0.20,...,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78,0
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,7817.0,7817.0,NaN,NaN,0.19,...,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,TRAIN_583,0,0.522340,2022-09-05 8:34,T050304,11920.0,11920.0,NaN,NaN,0.19,...,181.6,394.0,371.943396,353.0,51.71,59.64,54.61,57.05,63.18,0
584,TRAIN_584,0,0.519519,2022-09-05 11:09,T010305,NaN,NaN,NaN,NaN,NaN,...,184.9,466.0,448.634615,432.0,NaN,NaN,NaN,NaN,NaN,0
585,TRAIN_585,0,0.515214,2022-09-05 11:17,T010306,NaN,NaN,NaN,NaN,NaN,...,176.7,472.0,450.339623,432.0,NaN,NaN,NaN,NaN,NaN,0
594,TRAIN_594,0,0.524022,2022-09-08 22:38,T050304,14810.0,14810.0,NaN,NaN,0.19,...,168.7,384.0,369.811321,353.0,49.47,53.07,50.89,55.10,66.49,0


In [18]:
# # y_class 표현
# trainA_31_y_c = trainA_31['제품상태']
# trainT_31_y_c = trainT_31['제품상태']
# trainO_31_y_c = trainO_31['제품상태']

In [19]:
# y_class 표현
trainA_31_y_c = pd.get_dummies(trainA_31['Y_Class'].astype('str'))
trainT_31_y_c = pd.get_dummies(trainT_31['Y_Class'].astype('str'))
trainO_31_y_c = pd.get_dummies(trainO_31['Y_Class'].astype('str'))

In [20]:
# classification
trainA_31_y_c = torch.from_numpy(trainA_31_y_c.values)
trainT_31_y_c = torch.from_numpy(trainT_31_y_c.values)
trainO_31_y_c = torch.from_numpy(trainO_31_y_c.values)

# regression
trainA_31_y_r = torch.from_numpy(trainA_31['Y_Quality'].values)
trainT_31_y_r = torch.from_numpy(trainT_31['Y_Quality'].values)
trainO_31_y_r = torch.from_numpy(trainO_31['Y_Quality'].values)

## Adj 구축
- 같은 line끼리 edge 구축

In [22]:
line_A = trainA_31['LINE']
line_T = trainT_31['LINE']
line_O = trainO_31['LINE']

In [23]:
adj_A = np.array([[1 if i==j else 0 for i in line_A] for j in line_A])
adj_T = np.array([[1 if i==j else 0 for i in line_T] for j in line_T])
adj_O = np.array([[1 if i==j else 0 for i in line_O] for j in line_O])

In [24]:
print(adj_A.shape, adj_T.shape, adj_O.shape)

(249, 249) (343, 343) (6, 6)


## Model 구축

In [38]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# multi loss function
class Loss(nn.Module): 
    def __init__(self):
        super(Loss, self).__init__()
        self.loss = nn.CrossEntropyLoss(reduction='none')
    def forward(self, output, labels):
        labels = torch.argmax(labels, dim=1)
        loss = self.loss(output, labels)
        return torch.mean(loss)

# # binary loss function
# class Loss(nn.Module): 
#     def __init__(self):
#         super(Loss, self).__init__()
#         self.loss = nn.CrossEntropyLoss(reduction='none')
#         # self.loss = F.binary_cross_entropy()
#     def forward(self, output, labels):
#         # output = torch.argmax(output, dim=1)
#         # labels = torch.argmax(labels, dim=1)
#         labels = labels.unsqueeze(1).type(torch.FloatTensor)
#         # loss = self.loss(output, labels)
#         loss = F.binary_cross_entropy(output,labels)
#         return torch.mean(loss)

def sample_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
#    return np.array(mask, dtype=np.bool)
    return np.array(mask, dtype=bool)


def build_optimizer(model, lr, weight_decay):
    gcn1, gcn2 = [], []
    for name, p in model.named_parameters():
        if 'layer1' in name:
            gcn1.append(p)
        else:
            gcn2.append(p)
    opt = optim.Adam([{'params': gcn1, 'weight_decay': weight_decay},
                      {'params': gcn2}
                      ], lr=lr)
    return opt


def get_lr():
    pass


def get_loss(output, labels):
    loss = Loss()
    return loss(output, labels)

#multi
def get_accuracy(outputs, labels):
    outputs = torch.argmax(outputs, dim=1)
    labels = labels.argmax(labels, dim=1)
    outputs = outputs.numpy()
    labels = labels.numpy()
    correct = outputs == labels
    tp = np.sum(correct)
    return tp

# # binary
# def get_accuracy(outputs, labels):
#     outputs = torch.argmax(outputs, dim=1)
#     labels = labels.unsqueeze(1).type(torch.FloatTensor)
#     outputs = outputs.numpy()
#     labels = labels.numpy()
#     correct = outputs == labels
#     tp = np.sum(correct)
#     return tp

In [33]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Func


def preprocess_adj(A):
    '''
    Pre-process adjacency matrix
    :param A: adjacency matrix
    :return:
    '''
    I = np.eye(A.shape[0])
    A_hat = A + I # add self-loops
    D_hat_diag = np.sum(A_hat, axis=1)
    D_hat_diag_inv_sqrt = np.power(D_hat_diag, -0.5)
    D_hat_diag_inv_sqrt[np.isinf(D_hat_diag_inv_sqrt)] = 0.
    D_hat_inv_sqrt = np.diag(D_hat_diag_inv_sqrt)
    return np.dot(np.dot(D_hat_inv_sqrt, A_hat), D_hat_inv_sqrt)


class GCNLayer(nn.Module):
    def __init__(self, in_dim, out_dim, acti=True):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim) # bias = False is also ok.
        if acti:
            self.acti = nn.ReLU(inplace=True)
        else:
            self.acti = None
    def forward(self, F):
        output = self.linear(F)
        if not self.acti:
            return output
        return self.acti(output)


class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, p):
        super(GCN, self).__init__()
        self.gcn_layer1 = GCNLayer(input_dim, hidden_dim)
        # self.gcn_layer1 = GCNLayer(input_dim, input_dim)
        self.gcn_layer2 = GCNLayer(input_dim, input_dim)
        self.gcn_layer3 = GCNLayer(input_dim, hidden_dim)
        self.gcn_layer4 = GCNLayer(hidden_dim, num_classes, acti=False)
        self.dropout = nn.Dropout(p)

    def forward(self, A, X):
        A = torch.from_numpy(preprocess_adj(A)).float()
        X = self.dropout(X.float())
        F = torch.mm(A, X)
        F = self.gcn_layer1(F)
        F = self.dropout(F)
        F = torch.mm(A, F)
        # F = self.gcn_layer2(F)
        # F = self.dropout(F)
        # F = torch.mm(A, F)
        # F = self.gcn_layer3(F)
        # F = self.dropout(F)
        # F = torch.mm(A, F)
        output = self.gcn_layer4(F)
        # return output
        return torch.sigmoid(output)

In [34]:
# import argparse

# parser = argparse.ArgumentParser()
# # parser.add_argument('--dataset', type=str, default='citeseer', help='Dataset to train')
# parser.add_argument('--init_lr', type=float, default=0.01, help='Initial learing rate')
# parser.add_argument('--epoches', type=int, default=200, help='Number of traing epoches')
# parser.add_argument('--hidden_dim', type=list, default=16, help='Dimensions of hidden layers')
# parser.add_argument('--dropout', type=float, default=0.5, help='Dropout rate (1 - keep  probability)')
# parser.add_argument('--weight_decay', type=float, default=5e-4, help='Weight for l2 loss on embedding matrix')
# parser.add_argument('--log_interval', type=int, default=10, help='Print iterval')
# parser.add_argument('--log_dir', type=str, default='experiments', help='Train/val loss and accuracy logs')
# parser.add_argument('--checkpoint_interval', type=int, default=20, help='Checkpoint saved interval')
# parser.add_argument('--checkpoint_dir', type=str, default='checkpoints', help='Directory to save checkpoints')
# args = parser.parse_args()

import easydict

args = easydict.EasyDict({
    "init_lr": 0.01,
    "epoches": 200,
    "hidden_dim": 16,
    'dropout': 0.5,
    'weight_decay':5e-4,
    'log_interval':10,
    'checkpoint_interval':20
})

## Train

In [35]:
import os
import torch
import torch.nn.functional as F


def train(model,optimizer,adj,features,y_train):
        
    for epoch in range(args.epoches + 1):
        outputs = model(adj, features)
        loss = get_loss(outputs, y_train)

        # # binary
        # outputs = torch.sigmoid(outputs)
        # loss = get_loss(outputs, y_train)
        
        model.eval()
        outputs = model(adj, features)
        train_accuracy = get_accuracy(outputs, y_train)
        
        model.train()
        if epoch % args.log_interval == 0:
            print("Epoch: %d, train loss: %f, train ac: %f"
                  %(epoch, loss.detach().numpy(), train_accuracy))
        if epoch % args.checkpoint_interval == 0:
            torch.save(model.state_dict(), os.path.join('/content/drive/MyDrive/LGaimers/model/', "gcn_%d.pth"%epoch))
        optimizer.zero_grad()  # Important
        loss.backward()
        optimizer.step()

    return outputs

In [39]:
model_A = GCN(trainA_31_x.shape[1], args.hidden_dim, 3, args.dropout)
model_T = GCN(trainT_31_x.shape[1], args.hidden_dim, 3, args.dropout)
model_O = GCN(trainO_31_x.shape[1], args.hidden_dim, 3, args.dropout)

optimizer_A = build_optimizer(model_A, args.init_lr, args.weight_decay)
optimizer_T = build_optimizer(model_T, args.init_lr, args.weight_decay)
optimizer_O = build_optimizer(model_O, args.init_lr, args.weight_decay)

In [40]:
output_A = train(model_A,optimizer_A,adj_A,trainA_31_x,trainA_31_y_c)

TypeError: ignored

In [131]:
output_A.shape

torch.Size([249, 1])

In [133]:
output_A

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

In [72]:
output_A = torch.argmax(output_A, dim=1)
output_A

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0])

In [77]:
labels = torch.argmax(trainA_31_y_c, dim=1)
labels

tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0])